<a href="https://colab.research.google.com/github/AremyDiaz/Regresion_Logistica/blob/main/Actividad_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Actividad de regresión logística con variables categóricas**

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Se lee el archivo
df=pd.read_csv('BD_Socio formador (TrainingDataComplete).csv', index_col='Id')
df.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252000 entries, 1 to 252000
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Income             252000 non-null  int64 
 1   Age                252000 non-null  int64 
 2   Experience         252000 non-null  int64 
 3   Married/Single     252000 non-null  object
 4   House_Ownership    252000 non-null  object
 5   Car_Ownership      252000 non-null  object
 6   Profession         252000 non-null  object
 7   CITY               252000 non-null  object
 8   STATE              252000 non-null  object
 9   CURRENT_JOB_YRS    252000 non-null  int64 
 10  CURRENT_HOUSE_YRS  252000 non-null  int64 
 11  Risk_Flag          252000 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 25.0+ MB


##**Identificación de variables**

In [ ]:
df['Profession'].unique()

array(['Mechanical_engineer', 'Software_Developer', 'Technical_writer',
       'Civil_servant', 'Librarian', 'Economist', 'Flight_attendant',
       'Architect', 'Designer', 'Physician', 'Financial_Analyst',
       'Air_traffic_controller', 'Politician', 'Police_officer', 'Artist',
       'Surveyor', 'Design_Engineer', 'Chemical_engineer',
       'Hotel_Manager', 'Dentist', 'Comedian', 'Biomedical_Engineer',
       'Graphic_Designer', 'Computer_hardware_engineer',
       'Petroleum_Engineer', 'Secretary', 'Computer_operator',
       'Chartered_Accountant', 'Technician', 'Microbiologist',
       'Fashion_Designer', 'Aviator', 'Psychologist', 'Magistrate',
       'Lawyer', 'Firefighter', 'Engineer', 'Official', 'Analyst',
       'Geologist', 'Drafter', 'Statistician', 'Web_designer',
       'Consultant', 'Chef', 'Army_officer', 'Surgeon', 'Scientist',
       'Civil_engineer', 'Industrial_Engineer', 'Technology_specialist'],
      dtype=object)

In [ ]:
df['STATE'].value_counts()

Uttar_Pradesh        28400
Maharashtra          25562
Andhra_Pradesh       25297
West_Bengal          23483
Bihar                19780
Tamil_Nadu           16537
Madhya_Pradesh       14122
Karnataka            11855
Gujarat              11408
Rajasthan             9174
Jharkhand             8965
Haryana               7890
Telangana             7524
Assam                 7062
Kerala                5805
Delhi                 5490
Punjab                4720
Odisha                4658
Chhattisgarh          3834
Uttarakhand           1874
Jammu_and_Kashmir     1780
Puducherry            1433
Mizoram                849
Manipur                849
Himachal_Pradesh       833
Tripura                809
Uttar_Pradesh[5]       743
Chandigarh             656
Sikkim                 608
Name: STATE, dtype: int64

In [ ]:
df['CITY'].value_counts()

Vijayanagaram       1259
Bhopal              1208
Bulandshahr         1185
Saharsa[29]         1180
Vijayawada          1172
                    ... 
Ujjain               486
Warangal[11][12]     459
Bettiah[33]          457
Katni                448
Karaikudi            431
Name: CITY, Length: 317, dtype: int64

##**Análisis de correlación de variables dicotómicas**
###**Variable 1**

In [ ]:
#Se filtra la primera variable
var1 = df[(df['Profession'] == 'Comedian') | (df['Profession'] == 'Surgeon')]
var1.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
28,9643150,24,13,single,rented,no,Comedian,Indore,Madhya_Pradesh,6,14,0
153,3775604,46,11,single,rented,no,Surgeon,Proddatur,Andhra_Pradesh,5,11,1
160,1134496,77,3,single,rented,no,Surgeon,Jehanabad[38],Bihar,3,10,1
163,6847133,69,7,single,rented,no,Surgeon,Dhanbad,Jharkhand,6,10,0
247,2252443,77,0,single,rented,no,Surgeon,Chandigarh_city,Chandigarh,0,11,0


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = var1[['Income','Age','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS']]
Var_Dep = var1['Profession']

In [ ]:
#Redefinimos las variables
x = Vars_Indep
y = Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = None)

In [ ]:
#Se escalan los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "x" tanto de entrenamiento como de prueba, util
x_train = escalar.fit_transform(x_train)
x_test = escalar.transform(x_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(x_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(x_test)
y_pred

array(['Surgeon', 'Comedian', 'Comedian', ..., 'Comedian', 'Comedian',
       'Comedian'], dtype=object)

In [ ]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión')
print(matriz)

Matriz de confusión
[[811 264]
 [644 288]]


In [ ]:
#Calculo la precisión modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Comedian')
print('Modelo de precisión')
print(precision)

Modelo de precisión
0.5573883161512028


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5475834578973593


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label= 'Comedian')
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7544186046511628


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label= 'Comedian')
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.6411067193675889


###**Variable 2**

In [ ]:
#Se filtra la primera variable
var2 = df[(df['Profession'] == 'Computer_hardware_engineer') | (df['Profession'] == 'Software_Developer')]
var2.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
33,4128828,21,10,single,rented,no,Computer_hardware_engineer,Khammam,Telangana,10,12,0
57,5554586,39,18,single,rented,no,Software_Developer,Hindupur,Andhra_Pradesh,13,13,1
78,4994848,58,9,single,rented,no,Software_Developer,Durgapur,West_Bengal,9,13,0


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = var2[['Income','Age','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS']]
Var_Dep = var2['Profession']

In [ ]:
#Redefinimos las variables
x = Vars_Indep
y = Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = None)


In [ ]:
#Se escalan los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "x" tanto de entrenamiento como de prueba, util
x_train = escalar.fit_transform(x_train)
x_test = escalar.transform(x_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(x_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(x_test)
y_pred

array(['Computer_hardware_engineer', 'Computer_hardware_engineer',
       'Computer_hardware_engineer', ..., 'Computer_hardware_engineer',
       'Computer_hardware_engineer', 'Computer_hardware_engineer'],
      dtype=object)

In [ ]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión')
print(matriz)

Matriz de confusión
[[760 312]
 [590 423]]


In [ ]:
#Calculo la precisión modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Computer_hardware_engineer')
print('Modelo de precisión')
print(precision)

Modelo de precisión
0.562962962962963


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5673860911270984


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label= 'Software_Developer')
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.4175715695952616


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label= 'Software_Developer')
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.48398169336384445


###**Variable 3**

In [ ]:
#Se filtra la primera variable
var3 = df[(df['CITY'] == 'Bulandshahr') | (df['CITY'] == 'Vijayawada')]
var3.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
157,1013313,32,16,single,rented,no,Geologist,Bulandshahr,Uttar_Pradesh,10,12,0
281,2451457,31,9,married,rented,no,Politician,Bulandshahr,Uttar_Pradesh,9,12,0
555,340075,45,0,single,rented,no,Industrial_Engineer,Bulandshahr,Uttar_Pradesh,0,13,1
680,9099029,64,9,married,rented,yes,Chartered_Accountant,Bulandshahr,Uttar_Pradesh,7,10,0
737,960205,66,16,single,rented,no,Comedian,Vijayawada,Andhra_Pradesh,7,11,1


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = var3[['Income','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS']]
Var_Dep = var3['CITY']

In [ ]:
#Redefinimos las variables
x = Vars_Indep
y = Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = None)

In [ ]:
#Se escalan los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "x" tanto de entrenamiento como de prueba, util
x_train = escalar.fit_transform(x_train)
x_test = escalar.transform(x_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(x_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(x_test)
y_pred

array(['Vijayawada', 'Vijayawada', 'Vijayawada', 'Vijayawada',
       'Vijayawada', 'Vijayawada', 'Vijayawada', 'Vijayawada',
       'Vijayawada', 'Bulandshahr', 'Bulandshahr', 'Bulandshahr',
       'Bulandshahr', 'Vijayawada', 'Vijayawada', 'Bulandshahr',
       'Vijayawada', 'Bulandshahr', 'Bulandshahr', 'Bulandshahr',
       'Vijayawada', 'Bulandshahr', 'Vijayawada', 'Vijayawada',
       'Vijayawada', 'Vijayawada', 'Vijayawada', 'Bulandshahr',
       'Vijayawada', 'Vijayawada', 'Bulandshahr', 'Vijayawada',
       'Bulandshahr', 'Bulandshahr', 'Vijayawada', 'Bulandshahr',
       'Bulandshahr', 'Vijayawada', 'Bulandshahr', 'Bulandshahr',
       'Vijayawada', 'Bulandshahr', 'Vijayawada', 'Vijayawada',
       'Vijayawada', 'Vijayawada', 'Bulandshahr', 'Bulandshahr',
       'Bulandshahr', 'Vijayawada', 'Bulandshahr', 'Vijayawada',
       'Vijayawada', 'Bulandshahr', 'Bulandshahr', 'Vijayawada',
       'Bulandshahr', 'Bulandshahr', 'Bulandshahr', 'Bulandshahr',
       'Vijayawada', 'Bulan

In [ ]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión')
print(matriz)

Matriz de confusión
[[233 110]
 [169 196]]


In [ ]:
#Calculo la precisión modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Bulandshahr')
print('Modelo de precisión')
print(precision)

Modelo de precisión
0.5796019900497512


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6059322033898306


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Bulandshahr")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.6793002915451894


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label= "Bulandshahr")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.625503355704698


###**Variable 4**

In [ ]:
#Se filtra la primera variable
var4 = df[(df['CITY'] == 'Vijayanagaram') | (df['CITY'] == 'Bhopal')]
var4.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
219,7357515,50,13,single,rented,yes,Fashion_Designer,Bhopal,Madhya_Pradesh,7,14,0
233,1126863,43,1,single,rented,no,Mechanical_engineer,Vijayanagaram,Andhra_Pradesh,1,11,0
236,7167762,49,14,single,rented,no,Firefighter,Bhopal,Madhya_Pradesh,12,14,1
336,67428,49,12,single,rented,no,Scientist,Vijayanagaram,Andhra_Pradesh,10,11,0
439,9078403,25,18,married,rented,no,Official,Vijayanagaram,Andhra_Pradesh,9,12,0


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = var4[['Income','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS']]
Var_Dep = var4['CITY']

In [ ]:
#Redefinimos las variables
x = Vars_Indep
y = Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = None)

In [ ]:
#Se escalan los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "x" tanto de entrenamiento como de prueba, util
x_train = escalar.fit_transform(x_train)
x_test = escalar.transform(x_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(x_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(x_test)
y_pred

array(['Bhopal', 'Vijayanagaram', 'Vijayanagaram', 'Bhopal', 'Bhopal',
       'Bhopal', 'Vijayanagaram', 'Vijayanagaram', 'Vijayanagaram',
       'Vijayanagaram', 'Bhopal', 'Vijayanagaram', 'Vijayanagaram',
       'Vijayanagaram', 'Bhopal', 'Bhopal', 'Bhopal', 'Bhopal', 'Bhopal',
       'Vijayanagaram', 'Bhopal', 'Bhopal', 'Vijayanagaram', 'Bhopal',
       'Bhopal', 'Bhopal', 'Vijayanagaram', 'Vijayanagaram',
       'Vijayanagaram', 'Vijayanagaram', 'Vijayanagaram', 'Bhopal',
       'Vijayanagaram', 'Bhopal', 'Bhopal', 'Bhopal', 'Vijayanagaram',
       'Bhopal', 'Vijayanagaram', 'Bhopal', 'Vijayanagaram', 'Bhopal',
       'Bhopal', 'Vijayanagaram', 'Bhopal', 'Vijayanagaram',
       'Vijayanagaram', 'Vijayanagaram', 'Vijayanagaram', 'Bhopal',
       'Bhopal', 'Bhopal', 'Bhopal', 'Vijayanagaram', 'Vijayanagaram',
       'Bhopal', 'Vijayanagaram', 'Bhopal', 'Bhopal', 'Bhopal',
       'Vijayanagaram', 'Bhopal', 'Bhopal', 'Bhopal', 'Bhopal',
       'Vijayanagaram', 'Vijayanagaram', 'Vijayan

In [ ]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión')
print(matriz)

Matriz de confusión
[[205 141]
 [132 263]]


In [ ]:
#Calculo la precisión modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Bhopal')
print('Modelo de precisión')
print(precision)

Modelo de precisión
0.6083086053412463


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.631578947368421


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Bhopal")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5924855491329479


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label= "Bhopal")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.6002928257686676


###**Variable 5**

In [ ]:
#Se filtra la primera variable
var5 = df[(df['STATE'] == 'Andhra_Pradesh') | (df['STATE'] == 'West_Bengal')]
var5.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
12,6623263,22,4,single,rented,no,Designer,Adoni,Andhra_Pradesh,4,14,0
16,5694236,39,2,married,rented,yes,Economist,Anantapuram[24],Andhra_Pradesh,2,10,0
17,7315840,71,8,married,rented,no,Air_traffic_controller,Kamarhati,West_Bengal,8,14,0
20,5431918,40,1,single,rented,no,Artist,Amaravati,Andhra_Pradesh,1,14,0
24,9236505,36,19,single,rented,no,Chemical_engineer,Ongole,Andhra_Pradesh,6,14,0


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = var5[['Income','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS']]
Var_Dep = var5['STATE']

In [ ]:
#Redefinimos las variables
x = Vars_Indep
y = Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = None)

In [ ]:
#Se escalan los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "x" tanto de entrenamiento como de prueba, util
x_train = escalar.fit_transform(x_train)
x_test = escalar.transform(x_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(x_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(x_test)
y_pred

array(['Andhra_Pradesh', 'West_Bengal', 'Andhra_Pradesh', ...,
       'West_Bengal', 'Andhra_Pradesh', 'Andhra_Pradesh'], dtype=object)

In [ ]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión')
print(matriz)

Matriz de confusión
[[7064  456]
 [6658  456]]


In [ ]:
#Calculo la precisión modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'West_Bengal')
print('Modelo de precisión')
print(precision)

Modelo de precisión
0.5


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5138718053847205


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="West_Bengal")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.06409895979758223


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label= "West_Bengal")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.1136307002242711


###**Variable 6**

In [ ]:
#Se filtra la primera variable
var6 = df[(df['STATE'] == 'Uttar_Pradesh') | (df['STATE'] == 'Maharashtra')]
var6.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
6,6915937,64,0,single,rented,no,Civil_servant,Jalgaon,Maharashtra,0,12,0
18,3666346,56,12,single,rented,no,Politician,Bhusawal,Maharashtra,12,11,1
35,9086933,55,7,single,rented,no,Air_traffic_controller,Saharanpur,Uttar_Pradesh,7,13,0
37,9625415,77,15,married,rented,no,Secretary,Amravati,Maharashtra,9,10,0


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = var6[['Income','Age','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS']]
Var_Dep = var6['STATE']

In [ ]:
#Redefinimos las variables
x = Vars_Indep
y = Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = None)

In [ ]:
#Se escalan los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "x" tanto de entrenamiento como de prueba, util
x_train = escalar.fit_transform(x_train)
x_test = escalar.transform(x_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(x_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(x_test)
y_pred

array(['Maharashtra', 'Uttar_Pradesh', 'Uttar_Pradesh', ...,
       'Uttar_Pradesh', 'Uttar_Pradesh', 'Uttar_Pradesh'], dtype=object)

In [ ]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión')
print(matriz)

Matriz de confusión
[[1408 6306]
 [1462 7013]]


In [ ]:
#Calculo la precisión modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average = 'binary', pos_label = 'Maharashtra')
print('Modelo de precisión')
print(precision)

Modelo de precisión
0.4905923344947735


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5201680153190438


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Maharashtra")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.18252527871402643


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label= "Maharashtra")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.2660619803476946
